In [ ]:
import sys
import seaborn as sns
sys.path.append("../")
from packages.plotter import *
from packages.physicalObjects.physicalValues import *
from packages.BEMPP.singleTraceManager import SingleTraceManager

In [ ]:
#pip install numpy==1.20.3

In [ ]:
#pip install seaborn

In [ ]:
# Parameters:
radius = 0.0001
spacingMultiplier = 8
L = radius * spacingMultiplier
numberOfBubblesPerSide = 10
hFactor = 1/6
ka = 0.011261
frequency = ((ka/radius) * 1480) / (2*pi)
fs=12

In [ ]:
manager = SingleTraceManager(formulationType=2, logs=True)
# 0 = Dirichlet Formulation
# 1 = Neumann Formulation
# 2 = PMCHWT Formulation
# 3 = Muller Formulation


initialYCoordinate = 0 - L*(numberOfBubblesPerSide-1)/2 
initialZCoordinate = 0 - L*(numberOfBubblesPerSide-1)/2
for i in range(numberOfBubblesPerSide):
    for j in range(numberOfBubblesPerSide):
        yCoordinate = initialYCoordinate + i*L
        zCoordinate = initialZCoordinate + j*L
        manager.setNewBubble(center=(0,yCoordinate,zCoordinate), size=radius, h=hFactor*radius, intCoordinates=(0,i,j))

In [ ]:
manager.setCalculationParameters(frequency)

In [ ]:
#Scattering points
n = 100
r = 5* L*(numberOfBubblesPerSide-1)
omega_list = np.linspace(0, 2*pi, n)

In [ ]:
points1 = [np.array(([0], [r*np.sin(omega)], [r*np.cos(omega)])) for omega in omega_list]

In [ ]:
scatteredFields1 = list()
for point in points1:
    incidentField, scatteredField, totalField = manager.evaluateExteriorFieldOn(point)
    scatteredFields1.append(scatteredField)

In [ ]:
scatteredFields1 = [abs(x[0]) for x in scatteredFields1]
scatteredFields1 = np.array(scatteredFields1)
scatteredFields1 = scatteredFields1 * r

In [ ]:
points2 = [np.array(([r*np.sin(omega)], [r*np.cos(omega)], [0])) for omega in omega_list]

In [ ]:
scatteredFields2 = list()

for point in points2:
    incidentField, scatteredField, totalField = manager.evaluateExteriorFieldOn(point)
    scatteredFields2.append(scatteredField)

In [ ]:
scatteredFields2 = [abs(x[0]) for x in scatteredFields2]
scatteredFields2 = np.array(scatteredFields2)
scatteredFields2 = scatteredFields2 * r

In [ ]:
scatteredFields2 = np.array(scatteredFields2)

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.set_theme(style="darkgrid")
ax = fig.add_subplot(projection='polar')

ax.plot(omega_list, scatteredFields1,
        linewidth=3, linestyle='-',
        label='parallel',
       )
ax.plot(omega_list, scatteredFields2,
        linewidth=2, linestyle='--',
        label='perpendicular',
       )

ax.set_rlim([0, 0.02])
ax.set_rticks([0.005, 0.01, 0.015, 0.02])

plt.legend(loc='upper right', fontsize=fs)

ax.annotate("fundamental resonance",
            xy=(0.04, 0.88), xycoords='figure fraction',
            bbox=dict(boxstyle="square", facecolor="0.8", edgecolor='k'),
            fontsize=fs,
           )

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
         ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(fs)
plt.savefig('test.pdf', bbox_inches='tight')